<a href="https://colab.research.google.com/github/mposa/MEAM517/blob/master/Lecture13/van_der_pol_sos_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Drake

In [ ]:
# Install drake. We are using the lqr controller in drake as the final stabilizing controller.
# The installation process will take about 2 minutes but it's only required in the start of the Colab's virtual machine.
!curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/colab_drake_setup.py > colab_drake_setup.py
from colab_drake_setup import setup
setup()

<IPython.core.display.Javascript object>

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  199M  100  199M    0     0  82.8M      0  0:00:02  0:00:02 --:--:-- 82.8M
Reading package lists...
Building dependency tree...
Reading state information...
libfreetype6 is already the newest version (2.8.1-2ubuntu2).
libfreetype6 set to manually installed.
libglu1-mesa is already the newest version (9.0.0-2.1build1).
libglu1-mesa set to manually installed.
libjsoncpp1 is already the newest version (1.7.4-3).
libjsoncpp1 set to manually installed.
liblapack3 is already the newest version (3.7.1-4ubuntu1).
liblapack3 set to manually installed.
libogg0 is already the newest version (1.3.2-1).
libogg0 set to manually installed.
libtheora0 is already the newest version (1.1.1+dfsg.1-14).
libtheora0 set to manually installed.
libxt6 is already the newest version (1:1.1.5-1).
libxt6 set to manually installed.
python3-numpy is alread

## Setup example problem

In [ ]:
import math

import numpy as np
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
import pydrake.symbolic as sym
from pydrake.symbolic import Polynomial
from scipy.linalg import solve_lyapunov

# Initialize an empty optimization program.
prog = MathematicalProgram()

# Declare state "x" as indeterminates
x = prog.NewIndeterminates(2)

# Dynamics
a = 1
xdot = np.array([-a*(x[0] - x[0]**3 / 3 - x[1]),  -x[0]/a])

# Solve Lyapunov equation for V
A = np.array([[-a, a], [-1/a, 0]])
Q = np.identity(2);
#Q = np.array([[10, 0], [0, 1]])
S=solve_lyapunov(A.T, -Q)
V = x.T @ S @ x;
Vdot = V.Jacobian(x) @ xdot

rho = prog.NewContinuousVariables(1)
rho = rho[0]

p1 = (V - rho) * (x.T @ x) - Vdot

prog.AddSosConstraint(p1);

prog.AddCost(-rho);

In [ ]:
# Solve program
result = Solve(prog)
print(result.get_solution_result())

# Print resulting rho
print(result.GetSolution(rho))

SolutionResult.kSolutionFound
4.897068785762085
